# Interpolation and Decimation

Import libraries

In [1]:
import librosa
import matplotlib.pyplot as plt
import scipy
import soundfile
import numpy as np

Define interpolation

In [2]:
def interpolate(file, Fs, target_Fs, name, dir):
    factor = int(target_Fs/Fs)
    data = librosa.core.load(file, sr=Fs)[0]
    target_samples = int(len(data)*target_Fs/Fs)
    interp_data = np.zeros(target_samples)
    for i in range(len(data)):
        addr = int(i*target_samples/len(data))
        j=0
        while(addr<(i+1)*target_samples/len(data)):
            interp_data[addr] = data[i] 
            j += 1
            addr += 1
            
    filter = scipy.signal.firwin(51, Fs/2, fs=Fs*factor, pass_zero='lowpass')
    filtered_data = scipy.signal.lfilter(b=filter, a=filter[0] ,x=interp_data/100)
    soundfile.write(dir+'%s_interpolated_%s_to_%s.wav'%(name,Fs,target_Fs), filtered_data/10, target_Fs)
    return filtered_data/10

Define Decimation

In [7]:
def decimate(file, Fs, target_Fs,name, dir):
    factor = int(Fs/target_Fs)
    data = librosa.core.load(file, sr=Fs)[0]
    
    filter = scipy.signal.firwin(51, target_Fs/2, fs=Fs, pass_zero='lowpass')
    filtered_data = scipy.signal.lfilter(b=filter, a=filter[0] ,x=data/100)
    
    count = 0
    decim_data = np.zeros(int(target_Fs*len(data)/Fs))
    for i in range(len(data)):
        if i > count*Fs/target_Fs:
            if count<len(decim_data):
                decim_data[count] = filtered_data[i]
                count += 1
    
    soundfile.write(dir+'%s_decimated_%s_to_%s.wav'%(name, Fs,target_Fs), decim_data/10, target_Fs)
    return decim_data/10

Application

In [8]:
import os
path = '/Users/user/Library/CloudStorage/OneDrive-knu.ac.kr/20231/대학원_장길진_음성신호처리실습/ssp2023/lab01/segmented/'
folders = os.listdir(path)
for folder in folders:
    try: files = os.listdir(path + folder)
    except: continue
    for file in files:
        for item in [32000,48000,44100]:
            interpolate(path+folder+'/'+file,16000,item,file,'./interpolation_results/'+folder+'/')
            for item2 in [8000,11000]:
                decimate('./interpolation_results/'+folder+'/'+file+'_interpolated_16000_to_'+str(item)+'.wav',item,item2,file,'./decimation_results/'+folder+'/')

In [106]:
for item in [32000, 48000, 44100]:
    interpolate('kdigits0-3.wav', 16000, item)
for item in [32000, 48000, 44100]:
    for item2 in [8000, 11000]:
        decimate('interpolated_16000_%s.wav'%item,item,item2)